In [41]:
import numpy as np
import pandas as pd 
import keras
import re
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping,ModelCheckpoint
# fix random seed for reproducibility
np.random.seed(26)

In [20]:
work_train= {} 
work_test = {}
pre_process=["tfidf_tsvd_100.csv","w2v_100.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train[re.sub("\.csv","",f)] = pd.read_csv(path+"nw_working_train_"+f)
    work_test[re.sub("\.csv","",f)] = pd.read_csv(path+"nw_working_test_"+f)

In [8]:
y=pd.read_csv("../bases/new_training_variants.csv").Class.values - 1

In [9]:
magic_feat=pd.read_csv("../l2_meta_features/svd25_molecular_functions.csv")
magic_feat=magic_feat.drop(["Class","Gene","ID","Variation"],axis=1)
magic_train=magic_feat[:len(y)]
magic_test=magic_feat[len(y):]
magic_train=magic_train.reset_index(drop=True)
magic_test=magic_test.reset_index(drop=True)

In [22]:
ID_test=work_test["w2v_100"]["ID"]
ID_train=work_train["w2v_100"]["ID"]

In [28]:
deep_train=work_train["tfidf_tsvd_100"]#.merge(work_train["w2v_100"])
deep_train=deep_train.drop("ID",axis=1)
deep_test=work_test["tfidf_tsvd_100"]#.merge(work_test["w2v_100"])
deep_test=deep_test.drop("ID",axis=1)

In [29]:
deep_train_1=pd.concat((deep_train,magic_train),axis=1)
deep_test_1=pd.concat((deep_test,magic_test),axis=1)

In [ ]:
deep_train_1[]

In [48]:
deep_train_1.columns

Index(['Substitutions_var', 'Stop_codon_var', 'Fusion_var', 'gene_fusion_var',
       'Deletion_var', 'del_or_ins_var', 'Amplification_var', 'Truncation_var',
       'exon_var', 'frameshift_var',
       ...
       'molecular_SVD_16', 'molecular_SVD_17', 'molecular_SVD_18',
       'molecular_SVD_19', 'molecular_SVD_20', 'molecular_SVD_21',
       'molecular_SVD_22', 'molecular_SVD_23', 'molecular_SVD_24',
       'molecular_SVD_25'],
      dtype='object', length=162)

In [50]:
deep_test_1.columns

Index(['Substitutions_var', 'Stop_codon_var', 'Fusion_var', 'gene_fusion_var',
       'Deletion_var', 'del_or_ins_var', 'Amplification_var', 'Truncation_var',
       'exon_var', 'frameshift_var',
       ...
       'molecular_SVD_16', 'molecular_SVD_17', 'molecular_SVD_18',
       'molecular_SVD_19', 'molecular_SVD_20', 'molecular_SVD_21',
       'molecular_SVD_22', 'molecular_SVD_23', 'molecular_SVD_24',
       'molecular_SVD_25'],
      dtype='object', length=162)

In [56]:
deep_train_1.shape

(3689, 162)

In [62]:
# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=26)
cvscores = []
for train, test in kfold.split(np.array(deep_train_1), y):
  # create model
    model = Sequential()
    model.add(Dense(300, input_dim=162, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(300, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(300,activation="relu"))
    model.add(Dense(9, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    es=EarlyStopping(monitor="loss",mode="min",patience=10)
    checkpoint=ModelCheckpoint(filepath="checknn",save_best_only=True,monitor="val_loss",mode="min")
    model.fit(np.array(deep_train_1)[train], y_cat[train],epochs=100,batch_size=50,validation_split=0.2,callbacks=[es,checkpoint],verbose=True)
    model.load_weights("checknn")
    # evaluate the model
    scores = model.evaluate(np.array(deep_train_1)[test], y_cat[test])
    print("%s: %.2f" % (model.metrics_names[0], scores[0]))
    cvscores.append(scores[0])
print("%.2f (+/- %.2f)" % (np.mean(cvscores), np.std(cvscores)))

Train on 2356 samples, validate on 590 samples
Epoch 1/100
2356/2356 [==============================] - 1s - loss: 14.8663 - acc: 0.0764 - val_loss: 15.2747 - val_acc: 0.0508
Epoch 2/100
2356/2356 [==============================] - 0s - loss: 14.8540 - acc: 0.0781 - val_loss: 15.2748 - val_acc: 0.0508
Epoch 3/100
2356/2356 [==============================] - 0s - loss: 13.8030 - acc: 0.1430 - val_loss: 13.1721 - val_acc: 0.1814
Epoch 4/100
2356/2356 [==============================] - 0s - loss: 13.2248 - acc: 0.1791 - val_loss: 13.1729 - val_acc: 0.1814
Epoch 5/100
2356/2356 [==============================] - 0s - loss: 13.2246 - acc: 0.1795 - val_loss: 13.1733 - val_acc: 0.1814
Epoch 6/100
2356/2356 [==============================] - 0s - loss: 13.2243 - acc: 0.1795 - val_loss: 13.1739 - val_acc: 0.1814
Epoch 7/100
2356/2356 [==============================] - 0s - loss: 13.2243 - acc: 0.1795 - val_loss: 13.1746 - val_acc: 0.1814
Epoch 8/100
2356/2356 [==============================] - 

2361/2361 [==============================] - 0s - loss: 14.6777 - acc: 0.0894 - val_loss: 15.3121 - val_acc: 0.0491
Epoch 10/100
2361/2361 [==============================] - 0s - loss: 14.6777 - acc: 0.0894 - val_loss: 15.3131 - val_acc: 0.0491
Epoch 11/100
2361/2361 [==============================] - 0s - loss: 14.6776 - acc: 0.0894 - val_loss: 15.3138 - val_acc: 0.0491
Epoch 12/100
2361/2361 [==============================] - 0s - loss: 14.6777 - acc: 0.0894 - val_loss: 15.3141 - val_acc: 0.0491
Epoch 13/100
2361/2361 [==============================] - 0s - loss: 14.6777 - acc: 0.0894 - val_loss: 15.3140 - val_acc: 0.0491
Epoch 14/100
736/737 [============================>.] - ETA: 0sloss: 12.82
Train on 2363 samples, validate on 591 samples
Epoch 1/100
2363/2363 [==============================] - 2s - loss: 13.2572 - acc: 0.1756 - val_loss: 13.1497 - val_acc: 0.1827
Epoch 2/100
2363/2363 [==============================] - 0s - loss: 13.2226 - acc: 0.1786 - val_loss: 13.1508 - val_ac

2363/2363 [==============================] - 0s - loss: 13.2192 - acc: 0.1799 - val_loss: 13.1675 - val_acc: 0.1827
Epoch 58/100
2363/2363 [==============================] - 0s - loss: 13.2192 - acc: 0.1799 - val_loss: 13.1675 - val_acc: 0.1827
Epoch 59/100
2363/2363 [==============================] - 0s - loss: 13.2192 - acc: 0.1799 - val_loss: 13.1681 - val_acc: 0.1827
Epoch 60/100
576/735 [======================>.......] - ETA: 0sloss: 13.22
Train on 2364 samples, validate on 591 samples
Epoch 1/100
2364/2364 [==============================] - 2s - loss: 14.0604 - acc: 0.1261 - val_loss: 12.9852 - val_acc: 0.1929
Epoch 2/100
2364/2364 [==============================] - 0s - loss: 13.8183 - acc: 0.1413 - val_loss: 13.3156 - val_acc: 0.1726
Epoch 3/100
2364/2364 [==============================] - 0s - loss: 13.1820 - acc: 0.1810 - val_loss: 13.3148 - val_acc: 0.1726
Epoch 4/100
2364/2364 [==============================] - 0s - loss: 13.1808 - acc: 0.1823 - val_loss: 13.3172 - val_acc:

In [37]:
y_cat=to_categorical(y,9)